# Защита пользовательских данных путем их преобразования

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные таким образом, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('./datasets/insurance.csv')
df.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


## 2. Умножение матриц

В данном разделе необходимо ответить на следующий вопрос. Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

Исользуем обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$


**Ответ:** Качество линейно регрессии не изменится.

**Обоснование:** 
Пусть дана квадратная матрица **М**, причем $$detM \neq 0$$
Так как определитель матрицы **М** не равен нулю (т.е. матрица невырождена), то такая квадратная матрица является обратимой т.е. $$M M^{-1}=E$$

Домножим матрицу признаков **Х** на матрицу **M**. Пусть новые предсказания формируются на основе формулы:

$$
a_{new} = XMw, (1)
$$

Задача обучения:

$$
w_{new} = \arg\min_w MSE(XMw, y), (2)
$$

Формула обучения в таком случае:

$$
w_{new} = ((XM)^T XM)^{-1} (XM)^T y, (3)
$$

Для проверки результата подставим (3) в (1):

$$
a_{new} = XMw = XM ((XM)^T XM)^{-1} (XM)^T y = XM (XM)^{-1} ((XM)^T) ^{-1} (XM)^T y = E E y = y
$$


Таким образом даже в случае домножения матрицы признаков **Х** на квадратную обратимую матрицу **М**, при обучении линейной регресии по формуле (3) предсказания соответствуют вектору целевого признака, а значит качество линейной регрессии в данном случае не изменится.


## 3. Алгоритм преобразования

**Алгоритм**

1. Генерируется квадратная матрица **М** размерами: число столбцов в матрице признаков на число столбцов в матрице признаков;
2. Проверяется что детерминант сгенериованной матрицы не равен нулю;
3. Матица матрица признаков **X** умножается на матрицу **М** ;
4. Произведение матриц и есть искомая матрица с зашифрованными клиентскими данными.


**Обоснование**

Так как качество линейной регрессии не зависит от линейных преобраний матрицы **Х**, то можно домножать матицу признаков на любую обратимую матицу (проводить различные линеные преобразования). В следствии того что ядро пеобразования известно только генеирующему данные и оно случайно, данные окажутся зашифрованными, а качество модели не пострадает.

## 4. Проверка алгоритма

Создадим списки с целевой колонкой и колонками-признаками

In [5]:
y_col = ['Страховые выплаты']
x_col = df.columns.drop(y_col)
x_col

Index(['Пол', 'Возраст', 'Зарплата', 'Члены семьи'], dtype='object')

Сгенерируем случайную квадратную матицу с размерами равными длине списка x_col.

In [6]:
m = np.random.random((len(x_col), len(x_col)))
m

array([[0.99415528, 0.86994126, 0.57430311, 0.61282979],
       [0.59365589, 0.63852585, 0.06982195, 0.11262692],
       [0.39405186, 0.08078464, 0.27390643, 0.61621194],
       [0.33511726, 0.95137181, 0.77103913, 0.37251115]])

Убедимся что детеминант сгенерированной матрицы не равен нулю (что матрица обратима).

In [7]:
np.linalg.det(m)

-0.05714156764274234

Создадим датафрейм df_crypt данные в котором будут получены путем матричного произведения матрицы признаков исходного датасета и сгенерированной матрицы. Нумерацию индексов и колонок оставим как в исходном датасете, а так же присоединим к полученной матрице вектор целевого признака.

In [8]:
df_crypt = pd.DataFrame(df[x_col].values.dot(m), index=df.index, columns=x_col).join(df[y_col])
df_crypt

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,19570.641309,4034.919233,13589.966918,30569.715496,0
1,15001.613883,3100.140047,10412.427149,23421.607244,1
2,8292.305034,1714.994781,5754.059845,12943.717017,0
3,16445.099477,3384.031456,11424.906427,25699.148277,0
4,10302.370008,2127.227883,7151.487113,16086.898137,0
...,...,...,...,...,...
4995,14084.943921,2903.793271,9781.956606,22002.664998,0
4996,20668.836764,4255.776615,14355.841863,32293.707723,0
4997,13370.901330,2753.272705,9288.366459,20892.582482,0
4998,12900.555685,2659.429496,8961.163730,20154.338743,0


Видно что после проведенного преобразования извлечь личные данные из таблицы (не зная ядра преобразования) затруднительно.

Разделим выборку на тренировочную и тестовую.

In [9]:
train_idx, test_idx = train_test_split(df.index, test_size=0.2, random_state=42)

Убедимся что длины полученных выборок соответствуют тому что ожидалось.

In [10]:
len(train_idx)

4000

In [11]:
len(test_idx)

1000

In [12]:
(len(train_idx) + len(test_idx)) == len(df)

True

Для того чтобы убедиться что предложенное преобразование не влияет на качество линейной регрессии, обучим две модели: одну на на незашифрованных, другую на зашифрованных данных и оценим их качество на тестовых данных по метрике R2.

In [13]:
model = LinearRegression()
model.fit(df.loc[train_idx, x_col], df.loc[train_idx, y_col])
model.score(df.loc[test_idx, x_col], df.loc[test_idx, y_col])

0.43686949231379213

In [14]:
model_crypt = LinearRegression()
model_crypt.fit(df_crypt.loc[train_idx, x_col], df_crypt.loc[train_idx, y_col])
model_crypt.score(df_crypt.loc[test_idx, x_col], df_crypt.loc[test_idx, y_col])

0.43686949231371275

Как видно качество моделей практически идентично, что подтверждает работоспособность алгоритма.